<a href="https://colab.research.google.com/github/smsaladi/UniRep/blob/master/unirep_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## How to use the UniRep mLSTM "babbler". This version demonstrates the 64-unit and the 1900-unit architecture. 

We recommend getting started with the 64-unit architecture as it is easier and faster to run, but has the same interface as the 1900-unit one.

### Set up Colab environment

* Install aws cli
* Clone my fork of the Unirep repo
* Move into repo folder
* Weight download (below)
* Test data download (below)

In [0]:
! pip install awscli
! [[ ! -d /content/UniRep ]] && git clone https://github.com/smsaladi/UniRep.git

import os
os.chdir("/content/UniRep")

     |████████████████████████████████| 2.4MB 2.8MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 5.3MB 51.5MB/s 
  Found existing installation: rsa 4.0
    Uninstalling rsa-4.0:
      Successfully uninstalled rsa-4.0
  Found existing installation: botocore 1.13.4
    Uninstalling botocore-1.13.4:
      Successfully uninstalled botocore-1.13.4


Cloning into 'UniRep'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 127 (delta 0), reused 0 (delta 0), pack-reused 124
Receiving objects: 100% (127/127), 197.86 KiB | 7.61 MiB/s, done.
Resolving deltas: 100% (61/61), done.


Use the 64-unit or the 1900-unit model?

In [0]:
USE_FULL_1900_DIM_MODEL = True # if True use 1900 dimensional model, else use 64 dimensional one.

## Setup

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import pandas as pd

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

### Load Monkey patch `babbler.get_rep`

This lets `get_rep` work with a batch_size > 1 and use a given tf.Session (without having to initialize it on its own). `get_rep` computes on all of the turns all of the passed sequences into a single batch.

It won't work if they are of varying lengths, i.e. pad to the same length first. Sequences must also be passed as the values of a `pd.Series`. 

In [0]:
! [[ ! -f secy_test.faa ]] && wget -O secy_test.faa 'https://www.uniprot.org/uniprot/?query=database:(type:pfam%20pf00344)&format=fasta&force=true&sort=score&fil=reviewed:yes'
! pip install biopython
%run run_inference.py

--2019-11-05 02:45:59--  https://www.uniprot.org/uniprot/?query=database:(type:pfam%20pf00344)&format=fasta&force=true&sort=score&fil=reviewed:yes
Resolving www.uniprot.org (www.uniprot.org)... 193.62.193.81
Connecting to www.uniprot.org (www.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... No data received.
Retrying.

--2019-11-05 02:46:44--  (try: 2)  https://www.uniprot.org/uniprot/?query=database:(type:pfam%20pf00344)&format=fasta&force=true&sort=score&fil=reviewed:yes
Connecting to www.uniprot.org (www.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... No data received.
Retrying.

--2019-11-05 02:47:30--  (try: 3)  https://www.uniprot.org/uniprot/?query=database:(type:pfam%20pf00344)&format=fasta&force=true&sort=score&fil=reviewed:yes
Connecting to www.uniprot.org (www.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... No data received.
Retrying.

--2019-11-05 02:48:16--  (tr

Usage: run_inference.py [OPTIONS] SEQ_FN OUT_FN
Try "run_inference.py --help" for help.

Error: Missing argument "SEQ_FN".


SystemExit: ignored

Intialize babbler. `batch_size=1` doesn't actually do anything for the monkey patched `get_rep`

In [0]:
b = unirep.babbler1900(batch_size=1, model_path="./1900_weights")



Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API

Instructions for updating:
dim is deprecated, use axis instead
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all 

### Read test data and then run inference across a few conditions

In [0]:
seqs = series_from_seqio("secy_test.faa", "fasta")
max_len = len(max(seqs, key=len))
seqs = seqs.apply(lambda s: s.ljust(max_len, 'X'))


from timeit import default_timer as timer

def grouper(ser, n):
    ser_len = ser.size
    for i in range(0, ser_len, n):
        yield ser.iloc[i:(i+n)]

def batch_size_vs_time(func, iterable, grp_size):
  start = timer()
  [func(grp) for grp in grouper(iterable, grp_size)]
  end = timer()
  return end - start

In [7]:
# CPU only
with tf.Session() as sess:
    unirep.initialize_uninitialized(sess)
    def func(batch):
      b.get_rep(batch, sess)
    timing = [[gs, batch_size_vs_time(func, seqs, gs)] for gs in [16, 32, 64, 128, 256]]

timing

[[16, 546.6658579950001],
 [32, 311.33970561499973],
 [64, 215.62030503999995],
 [128, 162.09657374400012],
 [256, 135.92662289999998]]

In [0]:
# with GPU
with tf.Session() as sess:
    unirep.initialize_uninitialized(sess)
    def func(batch):
      b.get_rep(batch, sess)
    timing = [[gs, batch_size_vs_time(func, seqs, gs)] for gs in [16, 32, 64, 128, 256]]

timing

[[16, 26.899244273000022],
 [32, 13.71079222100002],
 [64, 8.502014164999991],
 [128, 6.426210394000009],
 [256, 4.381791538000016]]